In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
from multiprocessing import Pool

In [2]:
# Input files
weather_data_file = '../data/NOAA_weather_data/2015_california_only_all.csv'
fire_data_file = '../data/USDA_wildfire_data/california_fires.csv'

# Output files
weather_data_bins = '../data/spatial_data/NOAA_weather_data_bins.csv'
regridded_fire_data = '../data/USDA_wildfire_data/regridded_california_fires.csv'

In [3]:
weather = pd.read_csv(weather_data_file)
weather['ignition'] = 0

bins = weather[['lat','lon']]
unique_bins = bins.drop_duplicates()

unique_bins.to_csv(weather_data_bins, index=False)

In [4]:
fires = pd.read_csv(fire_data_file)

In [5]:
bin_array = np.column_stack([unique_bins['lon'], unique_bins['lat']])
fire_array = np.column_stack([fires['lon'], fires['lat']])

bin_tree = spatial.cKDTree(bin_array)
dist, indexes = bin_tree.query(fire_array)
indexes = pd.Series(indexes)

In [6]:
# Note: in testing, loop is faster than apply with lambda function by 3 tenths of a second

fire_bins = []

def index_loop():
    for index in indexes:
        fire_bins.append([unique_bins.iloc[index, 0], unique_bins.iloc[index, 1]])
        
%time index_loop()

fires[['lat', 'lon']] = fire_bins
fires.head()

CPU times: user 5.29 s, sys: 52.2 ms, total: 5.34 s
Wall time: 5.34 s


,lat,lon,date,time,size,size_class
0,39.93427,-121.1598,2005-02-02,1300.0,0.10,A
1,38.85770,-120.5208,2004-05-12,845.0,0.25,A
2,39.13992,-120.5874,2004-05-31,1921.0,0.10,A
3,38.62627,-120.0940,2004-06-28,1600.0,0.10,A
4,38.62627,-120.0940,2004-06-28,1600.0,0.10,A


In [7]:
fires.to_csv(regridded_fire_data, index=False)

In [8]:
# def add_ignition():
#     for index, row in fires.iterrows():
#         date = row['date']
#         lat = row['lat']
#         lon = row['lon']
#         weather.loc[(weather['time'] == date) & (weather['lat'] == lat) & (weather['lon'] == lon), 'ignition'] = 1

# %time add_ignition()

In [9]:
# %time fires.apply(lambda row: weather.loc[(weather[row['date']] == date) & (weather[row['lat']] == lat) & (weather[row['lon']] == lon), 'ignition'] = 1)